Importing all libraries and functions used in the Thesis

In [ ]:
import tsfresh
import os
import pandas as pd
from sklearn.impute import KNNImputer
import numpy as np
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, roc_auc_score,
matthews_corrcoef,recall_score 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgbm
import glob

AUs were extracted from OpenFace 2.0 software and CSV files with time series data were created. 
Each csv files corresponds to one video of donor at particular stage in donation process. 
Now, imputation of all the rows in csv file which has confidence lower than 0.85 is conducted using KNN Imputer.
This is crucial to ensure that no noise is present in the dataset with values of low confidence.

In [ ]:

folder_path = "confidential"


for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        id_num = re.search(r'video(\d+)_', filename).group(1)
        df = pd.read_csv(os.path.join(folder_path, filename))

        if len(df.columns) < 4:
            continue
        # Add ID and Frame columns which is needed for identification
        df['ID'] = id_num
        df['Frame'] = df.index + 1

        cols_to_impute = [col for col in df.columns if col.endswith('_r') or col == df.columns[3]]
        conf_mask = df.iloc[:, 3] < 0.85
        if conf_mask.all():
            print(f"Skipping file {filename} as all columns have less than 85% confidence")
            continue
        df.loc[conf_mask, cols_to_impute] = pd.np.nan
        
        imputer = KNNImputer(n_neighbors = 7)
        imputer.fit(df)
        
        imputed_values = pd.DataFrame(imputer.fit_transform(df[cols_to_impute]), columns = cols_to_impute)
        imputed_values = imputed_values.drop(' confidence', axis=1)
        imputed_values["ID"] = id_num
        imputed_values["Frame"] = df.index + 1

        # Saving new files
        new_filename = filename.replace(".csv", "_final_imputation.csv")
        imputed_values.to_csv(os.path.join(folder_path, new_filename), index=False)
###THIS CODE HAD TO DROP 186_6.csv and 190_6.csv files because they had no rows with confidence above 0.85

PLEASE NOTE THAT FROM HERE ON, PROCESS LOOKS EXATCLY THE SAME FOR TIME POINTS 4, 5 AND 6. THEREFORE, ONLY PROCESS FOR TIME POINT 4 IS INCLUDED IN THIS NOTEBOOK AS THE OTHERS CAN BE REPLICATED BY FOLLOING THESE STEPS

THE CODE BELOW MERGES ALL THE CSV FILES FOR 4th TIMESTAMP (RIGHT BEFORE NEEDLE INJECTION) INTO ONE DATAFRAME AND SAVES IT IN NEW CSV FILE

In [ ]:
folder_path =  "confidential"

num_files = len(glob.glob(os.path.join(folder_path, "*4_final_imputation.csv")))
print(num_files)
# New list for new dataframes
dfs = []

# Loop for files from 4th timepoint
for filename in os.listdir(folder_path):
    if filename.endswith("4_final_imputation.csv"):
        # Read the CSV file and append it to the list
        df = pd.read_csv(os.path.join(folder_path, filename))
        dfs.append(df)

# Merging dataframes in one large one
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head
# Saving dataframe to new csv file
merged_df.to_csv(os.path.join(folder_path, "Timepoint_4_merged.csv"), index=False)

THE CODE BELOW EXTRACTS SUM OF POINTS FROM VVR QUESTIONNAIRE FILE AT TIMEPOINT 4 AND CREATES BINARY CLASSIFICATION COLUMN 'At_Risk', which returns 1 when patient 
is At_Risk (score >11 out of 40) and 1 when patient is not At_Risk (score<11 out of 40)

In [ ]:
merged_df = pd.read_csv("Timepoint_4_merged.csv")


import pandas as pd
df = pd.read_csv('20221004_Time_points_final.csv', sep = ";", nrows=1989, encoding='latin1')

df = df[df['Time_point'] == 4]
df = df[['sum', 'ID']]


merged_df = pd.merge(merged_df, df, on='ID', how='left')
merged_df ['sum'] = merged_df ['sum'].fillna(0)  
merged_df ['At_Risk'] = merged_df ['sum'].apply(lambda x: 1 if x > 11 else 0)
merged_df = merged_df [['ID', 'sum', 'At_Risk'] + [col for col in merged_df .columns if col not in ['ID', 'sum', 'At_Risk']]]
merged_df = merged_df.reset_index(drop=True) 


final = merged_df[['ID', 'At_Risk']].copy()


y = final.groupby('ID').first().reset_index()
y.set_index('ID', inplace=True)
y = y['At_Risk'].squeeze()

# Drop unecessary columns:
merged_df = merged_df.drop(columns=['sum','At_Risk'])
merged_df

FEATURE EXTRACTION PROCEDURE FROM AUs DATAFRAME 

In [ ]:
from tsfresh.utilities.dataframe_functions import impute

A = extract_features(merged_df, column_id='ID', column_sort='Frame', default_fc_parameters=ComprehensiveFCParameters(),impute_function = impute)
                    
A =impute(A)
rows_with_nan = A[A.isna().any(axis=1)]

#Creating Final Dataframe With Features
X_final_A = A.dropna(subset=rows_with_nan.columns)
X_final_A= pd.DataFrame(X_final_A)

Selecting Relevant Features using select_features function in tsfresh

In [ ]:
X_filtered = select_features(X_final_A, y,ml_task = 'classification', fdr_level=0.275, hypotheses_independent=False)

In [ ]:
Machine Learning Process Implementation

In [ ]:

X = X_filtered
y = y




# Splitting data into training, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)

# Standardizing data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)
X_test_std = scaler.transform(X_test)

# Applying SMOTE on Train datasets
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_std, y_train)

#Setting Scoring for GridSearchCV for recall, as we want recall as the most important scoring method. 
scoring = make_scorer(recall_score)


# Random Forest Classifier with grid search cross-validation
rf = RandomForestClassifier(random_state=42)
param_grid_rf = {'n_estimators': [50, 100, 200,500],
                 'max_depth': [5, 10, 20, None],
                 'bootstrap': [True, False]}
rf_gs = GridSearchCV(rf, param_grid_rf, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), scoring=scoring)
rf_gs.fit(X_train_smote, y_train_smote)
print("Random Forest Classifier Best Parameters: ", rf_gs.best_params_)

# Non-Linear SVM Classifier with grid search cross-validation
svm = SVC(kernel='rbf', random_state=42)
param_grid_svm = {'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.1, 1, 10],
        'coef0': [0.0, 0.5, 1.0],
        'degree': [2, 3, 4]}
svm_gs = GridSearchCV(svm, param_grid_svm, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), scoring=scoring)
svm_gs.fit(X_train_smote, y_train_smote)
print("Non-Linear SVM Classifier Best Parameters: ", svm_gs.best_params_)


# LightGBM Classifier with grid search cross-validation
lgbm_clf = lgbm.LGBMClassifier(random_state=42)
param_grid_lgbm = {'n_estimators': [50, 100, 200],
                   'max_depth': [5, 10, 20, None],
                   'learning_rate': [0.01, 0.1, 0.5],
                   'num_leaves': [31, 50, 100, 200]}
lgbm_gs = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), scoring=scoring)
lgbm_gs.fit(X_train_smote, y_train_smote)
print("LightGBM Classifier Best Parameters: ", lgbm_gs.best_params_)




# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=rf_gs.best_params_['n_estimators'], max_depth=rf_gs.best_params_['max_depth'],
                             bootstrap =rf_gs.best_params_['bootstrap'], random_state=42)
rf_clf.fit(X_train_smote, y_train_smote)

# Non-Linear SVM Classifier
svm_clf = SVC(C=svm_gs.best_params_['C'], gamma=svm_gs.best_params_['gamma'],coef0=svm_gs.best_params_['coef0'], degree=svm_gs.best_params_['degree'],
              kernel='rbf', random_state=42)
svm_clf.fit(X_train_smote, y_train_smote)


# LightGBM Classifier
lgbm_clf = lgbm.LGBMClassifier(n_estimators=lgbm_gs.best_params_['n_estimators'], max_depth=lgbm_gs.best_params_['max_depth'], 
                               learning_rate =lgbm_gs.best_params_['learning_rate'], num_leaves =lgbm_gs.best_params_['num_leaves'],   random_state=42)
lgbm_clf.fit(X_train_smote, y_train_smote)


In [ ]:
TRAINING SET EVALUATION

In [ ]:
# Making predictions on train set
y_pred_rf = rf_clf.predict(X_train_std)
y_pred_svm = svm_clf.predict(X_train_std)
y_pred_lgbm = lgbm_clf.predict(X_train_std)
y_pred_linear_svm = linear_svm_gs.predict(X_train_std)

# Performance Metrics for Train Set
print("Random Forest Classifier Precision on Training Data: ", precision_score(y_train, y_pred_rf))
print("Random Forest Classifier Recall on Training Data: ", recall_score(y_train, y_pred_rf))
print("Random Forest Classifier F1-score on Training Data: ", f1_score(y_train, y_pred_rf))
print("Random Forest Classifier ROC AUC on Training Data: ", roc_auc_score(y_train, rf_clf.predict_proba(X_train_std)[:, 1]))
print("Random Forest Classifier PR AUC on Training Data: ", average_precision_score(y_train, rf_clf.predict_proba(X_train_std)[:, 1]))
print("Random Forest Classifier Matthews Coefficient on Training Data: ", matthews_corrcoef(y_train, y_pred_rf))

print("Non-Linear SVM Classifier Precision on Training Data: ", precision_score(y_train, y_pred_svm))
print("Non-Linear SVM Classifier Recall on Training Data: ", recall_score(y_train, y_pred_svm))
print("Non-Linear SVM Classifier F1-score on Training Data: ", f1_score(y_train, y_pred_svm))
print("Non-Linear SVM Classifier ROC AUC on Training Data: ", roc_auc_score(y_train, svm_clf.decision_function(X_train_std)))
print("Non-Linear SVM Classifier PR AUC on Training Data: ", average_precision_score(y_train, svm_clf.decision_function(X_train_std)))
print("Non-Linear SVM Classifier Matthews Coefficient on Training Data: ", matthews_corrcoef(y_train, y_pred_svm))

print("Linear SVM Classifier Precision on Training Data: ", precision_score(y_train, y_pred_linear_svm))
print("Linear SVM Classifier Recall on Training Data: ", recall_score(y_train, y_pred_linear_svm))
print("Linear SVM Classifier F1-score on Training Data: ", f1_score(y_train, y_pred_linear_svm))
print("Linear SVM Classifier ROC AUC on Training Data: ", roc_auc_score(y_train, linear_svm_clf.decision_function(X_train_std)))
print("Linear SVM Classifier PR AUC on Training Data: ", average_precision_score(y_train, linear_svm_clf.decision_function(X_train_std)))
print("Linear SVM Classifier Matthews Coefficient on Training Data: ", matthews_corrcoef(y_train, y_pred_linear_svm))

print("LightGBM Classifier Precision on Training Data: ", precision_score(y_train, y_pred_lgbm))
print("LightGBM Classifier Recall on Training Data: ", recall_score(y_train, y_pred_lgbm))
print("LightGBM Classifier F1-score on Training Data: ", f1_score(y_train, y_pred_lgbm))
print("LightGBM Classifier ROC AUC on Training Data: ", roc_auc_score(y_train, lgbm_clf.predict_proba(X_train_std)[:, 1]))
print("LightGBM Classifier PR AUC on Training Data: ", average_precision_score(y_train, lgbm_clf.predict_proba(X_train_std)[:, 1]))
print("LightGBM Classifier Matthews Coefficient on Training Data: ", matthews_corrcoef(y_train, y_pred_lgbm))

In [ ]:
VALIDATION SET EVALAUTION

In [ ]:
# Making predictions on validation set
y_pred_rf = rf_clf.predict(X_val_std)
y_pred_svm = svm_clf.predict(X_val_std)
y_pred_lgbm = lgbm_clf.predict(X_val_std)
y_pred_linear_svm = linear_svm_gs.predict(X_val_std)

# Performance Metrics of Validation Sets
print("Random Forest Classifier Precision on Validation Data: ", precision_score(y_val, y_pred_rf))
print("Random Forest Classifier Recall on Validation Data: ", recall_score(y_val, y_pred_rf))
print("Random Forest Classifier F1-score on Validation Data: ", f1_score(y_val, y_pred_rf))
print("Random Forest Classifier ROC AUC on Validation Data: ", roc_auc_score(y_val, rf_clf.predict_proba(X_val_std)[:, 1]))
print("Random Forest Classifier PR AUC on Validation Data: ", average_precision_score(y_val, rf_clf.predict_proba(X_val_std)[:, 1]))
print("Random Forest Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_val, y_pred_rf))

print("Non-Linear SVM Classifier Precision on Validation Data: ", precision_score(y_val, y_pred_svm))
print("Non-Linear SVM Classifier Recall on Validation Data: ", recall_score(y_val, y_pred_svm))
print("Non-Linear SVM Classifier F1-score on Validation Data: ", f1_score(y_val, y_pred_svm))
print("Non-Linear SVM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_val, svm_clf.decision_function(X_val_std)))
print("Non-Linear SVM Classifier PR AUC on Validation Data: ", average_precision_score(y_val, svm_clf.decision_function(X_val_std)))
print("Non-Linear SVM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_val, y_pred_svm))

print("Linear SVM Classifier Precision on Validation Data: ", precision_score(y_val, y_pred_linear_svm))
print("Linear SVM Classifier Recall on Validation Data: ", recall_score(y_val, y_pred_linear_svm))
print("Linear SVM Classifier F1-score on Validation Data: ", f1_score(y_val, y_pred_linear_svm))
print("Linear SVM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_val, linear_svm_clf.decision_function(X_val_std)))
print("Linear SVM Classifier PR AUC on Validation Data: ", average_precision_score(y_val, linear_svm_clf.decision_function(X_val_std)))
print("Linear SVM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_val, y_pred_linear_svm))

print("LightGBM Classifier Precision on Validation Data: ", precision_score(y_val, y_pred_lgbm))
print("LightGBM Classifier Recall on Validation Data: ", recall_score(y_val, y_pred_lgbm))
print("LightGBM Classifier F1-score on Validation Data: ", f1_score(y_val, y_pred_lgbm))
print("LightGBM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_val, lgbm_clf.predict_proba(X_val_std)[:, 1]))
print("LightGBM Classifier PR AUC on Validation Data: ", average_precision_score(y_val, lgbm_clf.predict_proba(X_val_std)[:, 1]))
print("LightGBM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_val, y_pred_lgbm))

In [ ]:
TEST SET EVALAUTION

In [ ]:

# Making predictions on validation data using best hyperparameters
y_pred_rf = rf_clf.predict(X_test_std)
y_pred_svm = svm_clf.predict(X_test_std)
y_pred_lgbm = lgbm_clf.predict(X_test_std)
y_pred_linear_svm = linear_svm_gs.predict(X_test_std)
# Evaluating performance on validation data
print("Random Forest Classifier Precision on Validation Data: ", precision_score(y_test, y_pred_rf))
print("Random Forest Classifier Recall on Validation Data: ", recall_score(y_test, y_pred_rf))
print("Random Forest Classifier F1-score on Validation Data: ", f1_score(y_test, y_pred_rf))
print("Random Forest Classifier ROC AUC on Validation Data: ", roc_auc_score(y_test, rf_clf.predict_proba(X_test_std)[:, 1]))
print("Random Forest Classifier PR AUC on Validation Data: ", average_precision_score(y_test, rf_clf.predict_proba(X_test_std)[:, 1]))
print("Random Forest Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_test, y_pred_rf))

print("Non-Linear SVM Classifier Precision on Validation Data: ", precision_score(y_test, y_pred_svm))
print("Non-Linear SVM Classifier Recall on Validation Data: ", recall_score(y_test, y_pred_svm))
print("Non-Linear SVM Classifier F1-score on Validation Data: ", f1_score(y_test, y_pred_svm))
print("Non-Linear SVM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_test, svm_clf.decision_function(X_test_std)))
print("Non-Linear SVM Classifier PR AUC on Validation Data: ", average_precision_score(y_test, svm_clf.decision_function(X_test_std)))
print("Non-Linear SVM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_test, y_pred_svm))

print("Linear SVM Classifier Precision on Validation Data: ", precision_score(y_test, y_pred_linear_svm))
print("Linear SVM Classifier Recall on Validation Data: ", recall_score(y_test, y_pred_linear_svm))
print("Linear SVM Classifier F1-score on Validation Data: ", f1_score(y_test, y_pred_linear_svm))
print("Linear SVM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_test, linear_svm_clf.decision_function(X_test_std)))
print("Linear SVM Classifier PR AUC on Validation Data: ", average_precision_score(y_test, linear_svm_clf.decision_function(X_test_std)))
print("Linear SVM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_test, y_pred_linear_svm))

print("LightGBM Classifier Precision on Validation Data: ", precision_score(y_test, y_pred_lgbm))
print("LightGBM Classifier Recall on Validation Data: ", recall_score(y_test, y_pred_lgbm))
print("LightGBM Classifier F1-score on Validation Data: ", f1_score(y_test, y_pred_lgbm))
print("LightGBM Classifier ROC AUC on Validation Data: ", roc_auc_score(y_test, lgbm_clf.predict_proba(X_test_std)[:, 1]))
print("LightGBM Classifier PR AUC on Validation Data: ", average_precision_score(y_test, lgbm_clf.predict_proba(X_test_std)[:, 1]))
print("LightGBM Classifier Matthews Coefficient on Validation Data: ", matthews_corrcoef(y_test, y_pred_lgbm))

In [ ]:
CREATING IMPORTANCE SCORES FOR FEATURES PER TIMEPOINT FOR EACH MODEL

In [ ]:
#RANDOM FOREST
model_rf = RandomForestClassifier(bootstrap = True, max_depth = 10, n_estimators = 100)
model_rf.fit(X_filtered, y)

sorted_indices_rf = importance_scores_rf.argsort()[::-1]
sorted_feature_names_rf = df_rf['Feature'].iloc[sorted_indices_rf]
sorted_importance_scores_rf = importance_scores_rf[sorted_indices_rf]

# Dataframe with name and importance score of the feature
df_sorted_rf = pd.DataFrame({'Feature': sorted_feature_names_rf, 'Importance Score': sorted_importance_scores_rf})
df_sorted_rf

In [ ]:
#Non-linear SVM
model_svm = SVC(kernel='rbf', C=10, coef0=0.0, degree=2, gamma='scale')  # Non-linear SVM with RBF kernel
model_svm.fit(X_filtered, y)


from sklearn.inspection import permutation_importance
result = permutation_importance(model_svm, X_filtered, y, n_repeats=10, random_state=42)


importance_scores_svm = result.importances_mean

sorted_indices_svm = importance_scores_svm.argsort()[::-1]
sorted_feature_names_svm = X_filtered.columns[sorted_indices_svm]
sorted_importance_scores_svm = importance_scores_svm[sorted_indices_svm]

# Dataframe with name and importance score of the feature
df_sorted_svm = pd.DataFrame({'Feature': sorted_feature_names_svm, 'Importance Score': sorted_importance_scores_svm})

In [ ]:
#LightGBM
model_lgb = lgb.LGBMClassifier(learning_rate=0.5, max_depth=10, n_estimators=100, num_leaves=31)
model_lgb.fit(X_filtered, y)

importance_scores_lgb = model_lgb.feature_importances_
normalized_importance_scores_lgb = importance_scores_lgb / importance_scores_lgb.sum()

sorted_indices_lgb = normalized_importance_scores_lgb.argsort()[::-1]
sorted_feature_names_lgb = X_filtered.columns[sorted_indices_lgb]
sorted_importance_scores_lgb = normalized_importance_scores_lgb[sorted_indices_lgb]

# Dataframe with name and importance score of the feature
df_sorted_lgb = pd.DataFrame({'Feature': sorted_feature_names_lgb, 'Importance Score': sorted_importance_scores_lgb})

ERROR ANALYSIS 

In [ ]:

import matplotlib.pyplot as plt


self_reported_df.drop_duplicates(subset='ID', inplace=True)

# Define the threshold for the binary classification
threshold = 11

# Convert 'X_test_std' array to a DataFrame
X_test_std = pd.DataFrame(X_test_std, index=y_test.index)

predictions_rf = rf_clf.predict(X_test_std)
predictions_non_linear_svm = svm_clf.predict(X_test_std)
predictions_lgbm = lgbm_clf.predict(X_test_std)

prediction_variables = ['predictions_rf', 'predictions_non_linear_svm', 'predictions_lgbm']

fig, axs = plt.subplots(1, 3, figsize=(12, 4)) 

# Creating three bar charts 
for i, variable in enumerate(prediction_variables):
    
    predictions = globals()[variable]
    plt.rcParams['font.family'] = 'Times New Roman'
    predictions_series = pd.Series(predictions, index=y_test.index, name='Prediction')

    merged_df = pd.merge(self_reported_df, predictions_series, left_on='ID', right_index=True, how='inner')

    # Calculate the counts of correct and incorrect predictions for each VVR score
    vvr_scores = merged_df['sum'].unique()
    correct_counts = []
    incorrect_counts = []

    for vvr_score in vvr_scores:
        df_subset = merged_df[merged_df['sum'] == vvr_score]
        correct_count = len(df_subset[df_subset['Prediction'] == df_subset['At_Risk']])
        incorrect_count = len(df_subset) - correct_count
        correct_counts.append(correct_count)
        incorrect_counts.append(incorrect_count)

    # Stacked Bar charts with correct predictions
    axs[i].bar(vvr_scores, correct_counts, color='gray', label='Correct Predictions', hatch='')

    # Stacked Bar charts with incorrect predictions on top of correct predictions
    axs[i].bar(vvr_scores, incorrect_counts, bottom=correct_counts, color='white', edgecolor='black',
               label='Incorrect Predictions', hatch='////')

    # Threshold line of 11
    axs[i].axvline(x=11, color='black', linestyle='dotted')
    
    # Titles for bar charts
    if variable == 'predictions_rf':
        title = 'Random Forest'
    elif variable == 'predictions_non_linear_svm':
        title = 'Non-linear SVM'
    elif variable == 'predictions_lgbm':
        title = 'LightGBM'

    axs[i].set_xlabel('VVR SCORES')
    axs[i].set_ylabel('Number of Donors')
    axs[i].set_title(f'{title}')
    axs[i].set_ylim(top=15)
    axs[i].legend()


plt.tight_layout()

 
plt.rcParams['font.family'] = 'Times New Roman'
# Save the plot as a JPEG image
plt.savefig('prediction_plot_timepoint4_test_set.jpeg', format='jpeg')
# Show the plot
plt.show()